# Pre-processing of data
The first thing to do is extracting the data we are interested in from the XML files (XML files are used from a dataset of US patent applications from 2001 to 2016. This dataset can be found here: https://figshare.com/articles/dataset/Chemical_reactions_from_US_patents_1976-Sep2016_/5104873). We are interested in the title, the general experimental procedure, the reaction SMILES, but most importantly the product SMILES.
Therefore, using xml.Etree in the following we iterate through the child elements of the root of the file. 


In [1]:
# First we import the necessary libraries

import xml.etree.cElementTree as et     # for parsing the XML file
import pandas as pd
import numpy as np
import os
import re

Now we use a function to iterate over all the XML files in the applications data from 2001. We collect the title, the paragraphText, mostly including the experimental procedures, the reaction SMILES and the product SMILES from it.

In [2]:
def extract_data(root_file):
    """function that extracts data from an XML file and returns a list of dictionaries containing the extracted information.
        Information to be extracted includes the title of the reaction, the experimental procedure, the reaction SMILES, and the product SMILES.

    Args:
        root_file: root of the parsed XML file

    Returns:
        reaction_list: list of dictionaries containing the extracted information
    """
    
    # Define the namespace that prevents mismatching of tags in the XML file
    ns = {'cml': 'http://www.xml-cml.org/schema', 'dl': 'http://bitbucket.org/dan2097'}

    # Create lists to store extracted information
    reaction_list = []

    # Find all <reaction> elements
    reaction_elements = root_file.findall('.//cml:reaction', ns)

    # Iterate over each <reaction> element
    for reaction_element in reaction_elements:
        # Create a dictionary to store information about the reaction
        reaction_dict = {}

        # Extract title
        title = reaction_element.find('.//dl:headingText', ns)
        if title is not None:
            reaction_dict['title'] = title.text

        # Extract paragraph text
        paragraph_text = reaction_element.find('.//dl:paragraphText', ns)
        if paragraph_text is not None:
            reaction_dict['paragraphText'] = paragraph_text.text

        # Extract reaction SMILES
        reaction_smiles = reaction_element.find('.//dl:reactionSmiles', ns)
        if reaction_smiles is not None:
            reaction_dict['reactionSmiles'] = reaction_smiles.text

        # Extract product SMILES
        product_elements = reaction_element.findall('.//cml:product', ns)
        product_smiles = []
        for product_element in product_elements:
            identifier_element = product_element.find('.//cml:identifier[@dictRef="cml:smiles"]', ns)
            if identifier_element is not None:
                smiles_value = identifier_element.get('value')
                if smiles_value is not None:
                    product_smiles.append(smiles_value)
        if product_smiles:
            reaction_dict['productSmiles'] = product_smiles

        # Append the reaction dictionary to the reaction list
        reaction_list.append(reaction_dict)

    # Check if any values were extracted
    #print("Reaction List:", reaction_list)
    return reaction_list


In a second step, we iterate through all files in every folder of the application data of the dataset and extract the data with the function extract_data. This takes quite a bit, but you can see the progress whenever a new folder is treated. 

In [3]:
Applications_list = []

# define path to access first XML file in the folder 2001 of applications
# Insert the path to the folder containing the XML files
for folder in os.listdir(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\applications'):
    folder = os.path.join(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\applications', folder)
    print(folder) 
    for file in os.listdir(folder):
        if file.endswith('.xml'):
            file = os.path.join(folder, file)
            tree = et.parse(file)
         # define root of the XML file to iterate through the file
            root = tree.getroot()
            Applications_list.append(extract_data(root))    
                  

C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2001
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2002
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2003
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2004
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2005
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2006
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2007
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2008
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2009
C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2010


KeyboardInterrupt: 

This is just some snippets from previous try-outs, nothing particular to look at s far. Just decided to keep it for now.

In [ ]:
# Applications_list = []
# # define path to access first XML file in the folder 2001 of applications
# for path in os.listdir(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2001'):
#     if path.endswith('.xml'):
#         path = os.path.join(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\applications\2001', path)
#         tree = et.parse(path)
        
#         # define root of the XML file to iterate through the file
#         root = tree.getroot()
#         Applications_list.append(extract_data(root))

In [ ]:
# # Get more familiar with the data and with how you can iterate through an XML file.
# count = 0
# for title in root:
#     print(title[0][0].tag)
#     count += 1
# print(count)

## Collect first values
Now that we are more familiar with the XML file and how we can iterate or access the different children of the root, we try to create a list with the values of interest.

In [ ]:
# Extract the title of the reactions
Title = []
for title in root.iter('{http://bitbucket.org/dan2097}headingText'):
    print(title.text)   # print the title of the reactions to see what they are like
    Title.append(title.text)
print(len(Title))

Dowicide (sodium o-phenylphenate)
Copolymer A. Preparation of Isooctyl Acrylate/Acrylamide/Vinyl Acetate (75/5/20) Copolymer
4-(Dodecyloxy)benzyl alcohol
4-(Dodecyloxy)benzyl bromide
4-(Dodecyloxy)benzyltripropargylammonium bromide (1)
Methyl (S)-2-(((benzyloxy)carbonyl)amino)-5-((t-butoxycarbonyl)amino)pentanoate
Preparation of DOPA-CDDP Cores:
Step D: 1-allylcyclopropane-1-sulfonvl chloride:
Step E: 1-allyl-N-(3,4-difluoro-2-(2-fluoro-4-iodophenylamino)phenyl)cyclopropane-1-sulfonamide:
Step A: 1-Allyl-cyclopropanesulfonic acid [3,4,6-trifluoro-2-(2-fluoro-4-iodo-phenylamino)phenyl]-amide:
Step a: Diethyl 2-methyl-3-oxopentanedioate:
Step d: 4-Chloro-1,5-dimethyl-6-oxo-1,6-dihydropyridine-3-carboxylic acid:
Step e: 4-(2-Fluoro-4-iodophenylamino)-1,5-dimethyl-6-oxo-1,6-dihydropyridine-3-carboxylic acid:
Step g: N-(4-(2-fluoro-4-iodophenylamino)-1,5-dimethyl-6-oxo-1,6-dihydropyridin-3-yl)cyclopropanesulfonamide:
Step 1: Methyl 3-amino-6-bromopyrazine-2-carboxylate
Step 3: 3-Amino-6-(4-

OOps! Already we encounter the first problem here: the list of titles only counts up to 32 instead of the 38 reactions we could previously extract from this first XML file! Therefore, it will not be possible to zip the different lists we will create in the following, as we would loose the information of the matching. Stay tuned for the solution to this problem!

In [ ]:
# Extract the experimental procedure
ExpProcedure = []
for expprocedure in root.iter('{http://bitbucket.org/dan2097}paragraphText'):
    # print(title.text)
    ExpProcedure.append(expprocedure.text)
print(len(ExpProcedure))

4745


In [ ]:
# Extract the reaction SMILES
RxnSmiles = []
for smiles in root.iter('{http://bitbucket.org/dan2097}reactionSmiles'):
    # print(title.text)
    RxnSmiles.append(smiles.text)
print(len(RxnSmiles))

4745


Here we had a problem:
while in the other tags, the value we wanted to extract was directly linked, in the product tag there are different values and we only want to extract the product smiles identifier. Here we need to be very specific to prevent extracting other identifier smiles from the reactants or spectators tags. 

In [ ]:
PrdSmiles = []

# Define the namespace
ns = {'cml': 'http://www.xml-cml.org/schema'}

# Find all <reaction> elements
reaction_elements = root.findall('.//cml:reaction', ns)

# Iterate over each <reaction> element
for reaction_element in reaction_elements:
    # Find all <product> elements within the current <reaction> element
    product_elements = reaction_element.findall('.//cml:product', ns)
    # Iterate over each <product> element
    for product_element in product_elements:
        # Find all <identifier> elements within the current <product> element
        identifier_elements = product_element.findall('.//cml:identifier[@dictRef="cml:smiles"]', ns)
        # Iterate over each <identifier> element
        for identifier_element in identifier_elements:
            # Extract the value attribute (SMILES value)
            smiles_value = identifier_element.attrib.get('value')
            if smiles_value is not None:
                # Append the SMILES value to the list or process it as needed
                PrdSmiles.append(smiles_value)

# Check if any values were extracted
print("Product SMILES:", PrdSmiles)

Product SMILES: ['C1=CC=C(C=C1)C2=CC=CC=C2O', 'ClC1=C(C=CC=C1)C=1C=C(N(N1)C)OCC1=C(C=CC=C1OC)N1N=NN(C1=O)C', 'ClC1=CC=C(C=C1)C1=NSC(=N1)OCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'S1C(=CC=C1)OCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'BrC1=CC=C(S1)OCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'ClC1=CC=C(C=C1)C1=CC=C(S1)OCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'COC1=C(C=CC=C1)C=1N=C(OC1)OCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'C1(=CC=CC=C1)C=1N=C(OC1)SCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'CN1N=C(C=C1C1=CC=C(C=C1)Cl)OCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'C1(=CC=CC=C1)C=1N=C(SC1)OCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'CN1N=C(C(=C1C1=CC=C(C=C1)Cl)C)OCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'ClC1=CC=C(C=C1)C1=NN=C(S1)OCC1=C(C=CC=C1C)N1N=NN(C1=O)C', 'BrC1=C(C(=CC=C1)N=C=O)C', 'COC1=C(C(=CC=C1)N=C=O)C', 'CC1=C(C=CC=C1Cl)N1N=NNC1=O', 'CC1=C(C=CC=C1Cl)N1N=NN(C1=O)C', 'BrCC1=C(C=CC=C1Cl)N1N=NN(C1=O)C', 'CC1=C(C=CC=C1OC)N1N=NNC1=O', 'CC1=C(C=CC=C1OC)N1N=NN(C1=O)C', 'BrCC1=C(C=CC=C1OC)N1N=NN(C1=O)C', 'CC1=C(C=CC=C1S[Si](C(C)C)(C(C)C)C(C)C)N1N=NN(C1=O)C', 'CC1=C(C=CC=C1SC)N1N=NN

In [ ]:
print(len(PrdSmiles))

4784


Now, to prevent mismatching through handling of lists of different lengths, the best is to create a dictionnary for every reaction containing the values of interest as keys. After that, we can build a dataframe where every dictionnary is one entry in the df.

In [ ]:
#reaction_list = extract_data(root)
print(Applications_list[23][0])

{'title': 'HDDA—1,6-hexanediol diacrylate (available from UCB Chemicals Corp., Augustana, S.C.)', 'paragraphText': 'HPA—an isometric mixture of 2-hydroxypropyl acrylate and 3-hydroxypropyl acrylate (available as ROCRYL 430 from Rohm and Haas, Philadelphia, Pa.)', 'reactionSmiles': '[C:1]([O:5][CH2:6][CH:7](O)[CH3:8])(=[O:4])[CH:2]=[CH2:3].[C:10]([O:14][CH2:15][CH2:16][CH2:17]O)(=[O:13])[CH:11]=[CH2:12]>>[CH2:12]=[CH:11][C:10]([O:14][CH2:15][CH2:16][CH2:17][CH2:8][CH2:7][CH2:6][O:5][C:1]([CH:2]=[CH2:3])=[O:4])=[O:13].[C:10]([O:14][CH2:15][CH2:16][CH2:17][CH2:8][CH2:7][CH2:6][O:5][C:1](=[O:4])[CH:2]=[CH2:3])(=[O:13])[CH:11]=[CH2:12] |f:2.3|', 'productSmiles': ['C=CC(=O)OCCCCCCOC(=O)C=C.C(C=C)(=O)OCCCCCCOC(C=C)=O']}


# Put extracted data into a dataframe.
Now that we created a list of lists containing a dictionnary for every reaction that was extracted from the XML files, we put the lists into a dataframe.

In [ ]:
# Create a DataFrame from the extracted data
df_extracts = pd.DataFrame(Applications_list[0])

# Iterate through the list of dictionaries and create a DataFrame
for i in range(1, len(Applications_list)):
    df = pd.DataFrame(Applications_list[i])
    
    # Concatenate the DataFrames
    df_extracts = pd.concat([df_extracts, df], ignore_index=True)

print(df_extracts.shape)
df_extracts.head()


(1939253, 4)


,paragraphText,reactionSmiles,productSmiles,title
0,"PL 137,526 describes the hydrogenation of p-te...",[C:1]([C:5]1[CH:10]=[CH:9][C:8]([OH:11])=[CH:7...,[C(C)(C)(C)C1CCC(CC1)O],NaN
1,"Slurry aluminum chloride (140.9 g, 1.075 mol) ...",[Cl-].[Al+3].[Cl-].[Cl-].[Cl:5][CH2:6][CH2:7][...,[ClCCCC(=O)C1=CC=C(C=C1)C(C)C],Step h: 4-Chloro-1-(4-isopropyl-phenyl)-butan-...
2,"Suspend anhydrous AlCl3 (156 g, 1.15 mol) in t...",[Al+3].[Cl-].[Cl-].[Cl-].[Cl:5][CH2:6][CH2:7][...,[ClCCCC(=O)C1=CC=C(C=C1)C],Step d: 4-Chloro-1-(4-methyl-phenyl)-butan-1-one
3,Dissolve 4-chloro-1-(4-isopropyl-phenyl)-butan...,[Cl:1][CH2:2][CH2:3][CH2:4][C:5]([C:7]1[CH:12]...,[BrC(C)(C)C1=CC=C(C=C1)C(CCCCl)=O],1-[4-(1-Bromo-1-methyl-ethyl)-phenyl]-4-chloro...
4,Dissolve 4-chloro-1-(4-isopropyl-phenyl)-butan...,[Cl:1][CH2:2][CH2:3][CH2:4][C:5]([C:7]1[CH:12]...,[BrC(C)(C)C1=CC=C(C=C1)C(CCCCl)=O],1-[4-(1-Bromo-1-methyl-ethyl)-phenyl]-4-chloro...


That Dataframe is huge! Finally, save all the extracted data into a csv file on the local device.

In [ ]:
df_extracts.to_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\extracted_data_raw_applications.csv', index=False)

# Further Processing of the Data using Regex and LLM
Of course, not all entries in the data frame can be used for our model. Many of the experimental procedures do not include any information about the Rf value. Thus, we need to find the entries with Rf values. As all experimental procedures are written differently, we will try to find the value of interest by using Regex (Regular Expressions).

First, we load the extracted data into a new dataframe.

In [4]:
df_new = pd.read_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\extracted_data_raw_applications.csv')

And we create another dataframe, which only includes the first 1000 rows of the dataframe to reduce the cost of executing the test code below.

In [19]:
df_new.shape
df_new = df_new.iloc[0:1000]
df_new.shape

(1000, 4)

Seems to have worked! Now let's take a closer look at how we will try to extract the Rf values. 
Throughout different testing until now, we found several criteria that need to be defined in the Regex pattern for the Rf value: it should be a number following the general pattern 0.XY with Y not being obligatory and X being some digit between 2 and 8 (this is to exclude as many other matches as possible and as Rf values should ideally be around 0.5, we thought this would be optimal). Furthermore, the pattern should not be followed by other digits (e.g. 0.45005) nor include special signs (0.5:0.4) or temperature values. The last remaining problem is, to distinguish between information about quantity and the Rf value (e.g. 0.56 vs. 0.56 mg).

In [6]:
def extract_rf_eluent(Dataframe):
    """Function that applies defined regex patterns to data in a dataframe and creates 
    the following new columns: 
    Rf value, solvent A, solvent B, % solvent A, % solvent B. If no Rf value can be found, all columns
    are filled with NaN. 

    Args:
        Dataframe (_type_): Dataframe containing the extracted data from the US patents
    """
    # copy the dataframe to leave old dataframe unchanged
    df = Dataframe.copy()
    
    # Define the regex patterns
    Rf_check = r'( ?R[fF]?[ :=(]?)'
    Rf_pattern = r'(0\.(?!0|9)\d{1,2})\b(?! *mg\b| *mL\b| *g\b)' # exclude decimals that start by 0.0x, this yet seems not to work though, try to specifiy that digit after . cannot start with 0.
    #Rf_pattern = r'[ =:]?(0\.[^0][0-9])[^\dmglL](?! mg)' # exclude decimals that start by 0.0x, this yet seems not to work though, try to specifiy that digit after . cannot start with 0.
    
    #set a count to see how many multiple Rf values are found in the dataframe, how many NaN values are found
    count = 0
    count_nan = 0
    
    # Extract the Rf values from the paragraphText and put them into a new column
    for index, row in df.iterrows():
        checkRf = re.findall(Rf_check, row['paragraphText'])
        
        #check if Rf value can be found in the paragraphText column
        if checkRf:
             #try to find the Rf value in the paragraphText column
            match = re.findall(Rf_pattern, row['paragraphText'])
            if match:
                df.at[index, 'Rf_value'] = match[0] # df['paragraphText'].str.extract(Rf_pattern)
            
                # Check if multiple Rf values were found (potential error source)
                if len(match) > 1:
                    print('Multiple Rf values found in paragraphText:', match, 'at index:', index)
                    count += 1
       
        else:
            df.at[index, 'Rf_value'] = np.nan
            count_nan += 1
            
    print("Number of entries with multiple Rf values:", count)
    print("Number of entries with no Rf values found:", count_nan)
    print("Number of entries with Rf values found:", df['Rf_value'].count())
    return df
    

Turns out it would be very painful to do all this with regex. Not the best idea. Instead, we will try to use a LLM in the following. To still reduce the cost of computation, we will pre-filter the dataframe with the subsequent function to only get back data where an Rf value can be found in the experimental procedure. 

In [20]:
def extract_rows_with_rf(Dataframe):
    """Function that extracts rows with Rf values from a dataframe and returns a new dataframe containing only these rows. 

    Args:
        Dataframe (_type_): Dataframe containing the extracted data from the US patents
    """
    # copy the dataframe to leave old dataframe unchanged
    df = Dataframe.copy()
    
    # Define the regex patterns
    Rf_check = r'( ?R[fF]?[ :=(]?)'
    
    # List to store indices of rows without Rf values
    rows_to_drop = []
    
    # Search for rows with Rf values in the paragraphText column
    for index, row in df.iterrows():
        checkRf = re.findall(Rf_check, row['paragraphText'])

        if not checkRf:
            rows_to_drop.append(index)
               
    # Drop rows without Rf values
    df = df.drop(rows_to_drop)
            
    return df
    

That is still applying the first function and one can clearly see the mess. Won't be possible to extract the exact Rf value in the multiple detected rows.

In [7]:
df_new.head()
df_processed_first_try = extract_rf_eluent(df_new)
df_processed_first_try.head()
#df_processed_first_try.to_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\extracted_data_first_processing_rf_values.csv', index=False)

C:\Users\milen\AppData\Local\Temp\ipykernel_26032\3340503067.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.18' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Rf_value'] = match[0] # df['paragraphText'].str.extract(Rf_pattern)


Multiple Rf values found in paragraphText: ['0.57', '0.35', '0.56'] at index: 232
Multiple Rf values found in paragraphText: ['0.3', '0.55', '0.37', '0.27', '0.30', '0.3', '0.37', '0.27', '0.37', '0.10'] at index: 236
Multiple Rf values found in paragraphText: ['0.46', '0.3'] at index: 237
Multiple Rf values found in paragraphText: ['0.4', '0.3'] at index: 267
Multiple Rf values found in paragraphText: ['0.4', '0.3'] at index: 268
Multiple Rf values found in paragraphText: ['0.41', '0.1', '0.41', '0.32'] at index: 281
Multiple Rf values found in paragraphText: ['0.58', '0.64', '0.58'] at index: 287
Multiple Rf values found in paragraphText: ['0.1', '0.6'] at index: 291
Multiple Rf values found in paragraphText: ['0.85', '0.85'] at index: 300
Multiple Rf values found in paragraphText: ['0.5', '0.1', '0.15'] at index: 382
Multiple Rf values found in paragraphText: ['0.5', '0.1'] at index: 383
Multiple Rf values found in paragraphText: ['0.2', '0.82'] at index: 412
Multiple Rf values foun

,paragraphText,reactionSmiles,productSmiles,title,Rf_value
0,"PL 137,526 describes the hydrogenation of p-te...",[C:1]([C:5]1[CH:10]=[CH:9][C:8]([OH:11])=[CH:7...,['C(C)(C)(C)C1CCC(CC1)O'],NaN,NaN
1,"Slurry aluminum chloride (140.9 g, 1.075 mol) ...",[Cl-].[Al+3].[Cl-].[Cl-].[Cl:5][CH2:6][CH2:7][...,['ClCCCC(=O)C1=CC=C(C=C1)C(C)C'],Step h: 4-Chloro-1-(4-isopropyl-phenyl)-butan-...,NaN
2,"Suspend anhydrous AlCl3 (156 g, 1.15 mol) in t...",[Al+3].[Cl-].[Cl-].[Cl-].[Cl:5][CH2:6][CH2:7][...,['ClCCCC(=O)C1=CC=C(C=C1)C'],Step d: 4-Chloro-1-(4-methyl-phenyl)-butan-1-one,NaN
3,Dissolve 4-chloro-1-(4-isopropyl-phenyl)-butan...,[Cl:1][CH2:2][CH2:3][CH2:4][C:5]([C:7]1[CH:12]...,['BrC(C)(C)C1=CC=C(C=C1)C(CCCCl)=O'],1-[4-(1-Bromo-1-methyl-ethyl)-phenyl]-4-chloro...,NaN
4,Dissolve 4-chloro-1-(4-isopropyl-phenyl)-butan...,[Cl:1][CH2:2][CH2:3][CH2:4][C:5]([C:7]1[CH:12]...,['BrC(C)(C)C1=CC=C(C=C1)C(CCCCl)=O'],1-[4-(1-Bromo-1-methyl-ethyl)-phenyl]-4-chloro...,NaN


Now the second function: already the dataframe size cut down from 1000 entries to 246 which we can hopefully treat with a LLM.

In [22]:
df_processed_second_try = extract_rows_with_rf(df_new)
df_processed_second_try.head()
df_processed_second_try.shape

(246, 4)

For the LLM we use the API of the open source model offered by Groq Clouds (https://console.groq.com/docs/quickstart). Different models can be tested out.

In [14]:
#Access token croq: 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

from groq import Groq

# Create a Groq client (it is recommended to use the following Quickstart: https://console.groq.com/docs/quickstart)
# However, this did not work in our case and we had to use the following code to create a client 
client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Give me the Rf value, the solvent mixture and their ratio of the following procedure: Rf(Hex/EtOAc 1:20):0.22 in the following format: Rf= , solvent A= , solvent B= , % solvent A= , % solvent B= .",
        }
    ],
    model="mixtral-8x7b-32768", # other models: LlaMA3 70 b (llama3-70b-8192) can be found here: https://console.groq.com/docs/models
)

print(chat_completion.choices[0].message.content)

Rf = 0.22
Solvent A (Hexane) = 1
Solvent B (Ethyl Acetate) = 20
% solvent A = 4.76%
% solvent B = 95.24%

Note: The % solvent A and % solvent B are calculated by dividing each solvent ratio by the total sum of the solvent ratio.
